In [1]:
from selenium import webdriver # requres sslenium to webscrape, i used the firefox driver. 
import pandas as pd  


We need to get the predicted fantasy points per game. This comes from the courtesy of http://rotoguru1.com/cgi-bin/fstats.cgi

We need to understand their API in order to create good enough queries so we can scrape them with selenium

The api seems to follow: week =1, game=fanduel, fd


we need to parse positions: 

In [2]:
driver = webdriver.Firefox()
# lets get week one, fanduel
driver.get('http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd') 


table = driver.find_element_by_xpath('/html/body/table/tbody/tr/td[3]/p[6]/table/tbody/tr/td[1]/table')
#('/html/body/center[1]/font/form/table/tbody/tr[1]/td[4]/input')

NoSuchWindowException: Message: Browsing context has been discarded


In [ ]:
allRows =table.find_elements_by_xpath('/html/body/table/tbody/tr/td[3]/p[6]/table/tbody/tr/td[1]/table/tbody/tr')

So we need to make a pandas DF from this list. its probably better if we make different DataFrames for each role so we can see how they do over time in comparison, then join them if we need to, for the puposes of modelling. 

we're going to loop thru the scraped text of each of the rows, of each week, of each year. Sigh. gotta love text parsing. 
We'll make a list of positions: QB. RB, WR, TE, PK, Def. Pk = kickers.

Each time we hit the seperator, we increment the indexor and create a new Dataframe. 

Each dataframe will be named after a position. the columns will be: name ,team, opponent,points, FD salary




In [ ]:
#make named DataFrames
QB = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
RB = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
WR = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
TE = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
PK = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
Def = pd.DataFrame(columns=['name' ,'team', 'opponent','points', 'salary','week'])
# put Dataframes in list
DataFrames = [ QB,RB,WR,TE,PK,Def]


currentPosition = -1
skipNext = False
index = 0
for row in allRows:
    if skipNext == True:
        skipNext = False 
        continue
    if row.text == "Jump to:     QB   |   RB   |   WR   |   TE   |   PK   |   Def   |": 
        currentPosition += 1
        skipNext = True
        index = 0

    else:
        #add to list, legitimate player
        nameList = row.find_elements_by_xpath('td')
        name = nameList[0].text
        team = nameList[1].text
        opponent = nameList[2].text[2:]
        points = nameList[3].text
        salary = nameList[4].text[1:]
        week = 1
        if salary == "/A":
            salary = 0
        else:
            salary = nameList[4].text[1:]

            salary=int(salary.replace(',',''))
        
        
        appending = [name,team,opponent,points, salary, week]
        
        DataFrames[currentPosition].loc[index] = appending
        
        index += 1 

In [ ]:
nameList = allRows[2].find_elements_by_xpath('td')
name = nameList[0]

team = nameList[1].text
opponent = nameList[2].text[2:]
points = nameList[3]
salary = nameList[4].text[1:]
salary=int(salary.replace(',',''))

print(team)

In [ ]:
for i in allRows[2].find_elements_by_xpath('td'):
    print(i.text)

In [ ]:
DataFrames[4]

In [ ]:
driver.close()

In [9]:
qbStats = pd.read_csv('./NFL-Statistics-Scrape/Game_Logs_Quarterback.csv')
qbY = pd.read_csv('QB.csv',index_col=0)
qbStats = qbStats.rename({'Name':'name','Week':'week', 'Opponent':'opponent'},axis='columns')


basicQBStats = pd.read_csv('./NFL-Statistics-Scrape/Basic_Stats.csv')

basicQBStats = basicQBStats.rename({'Name':'name'}, axis='columns')


qbStats2019 = qbStats[qbStats['Year']==2019]



qbY['opponent'] =[x.upper() for x in qbY['opponent']]
#qbY['']

gameY = pd.merge(right = qbY , left = qbStats2019,how='inner') 

#basicQBStats

#qbY['weeks']

gameY
#qbStats2019


,Player Id,name,Position,Year,Season,week,Game Date,Home or Away,opponent,Outcome,...,Passer Rating,Rushing Attempts,Rushing Yards,Yards Per Carry,Rushing TDs,Fumbles,Fumbles Lost,team,points,salary
0,lamarjackson/2560757,"Jackson, Lamar",QB,2019,Regular Season,1,09/08,Away,MIA,W,...,158.3,3,6,2.0,0,--,--,bal,33.56,7400
1,lamarjackson/2560757,"Jackson, Lamar",QB,2019,Regular Season,5,10/06,Away,PIT,W,...,54.9,14,70,5.0,0,1,0,bal,14.44,8300
2,lamarjackson/2560757,"Jackson, Lamar",QB,2019,Regular Season,7,10/20,Away,SEA,W,...,69.4,14,116,8.3,1,2,0,bal,23.32,8400
3,deshaunwatson/2558063,"Watson, Deshaun",QB,2019,Regular Season,3,09/22,Away,LAC,W,...,135.8,7,18,2.6,0,1,1,hou,25.84,8200
4,deshaunwatson/2558063,"Watson, Deshaun",QB,2019,Regular Season,7,10/20,Away,IND,L,...,81.5,3,32,10.7,0,--,--,hou,17.52,8300
5,blakebortles/2543477,"Bortles, Blake",QB,2019,Regular Season,7,10/20,Away,ATL,W,...,0.0,--,--,--,--,--,--,lar,0.00,6000
6,mattryan/310,"Ryan, Matt",QB,2019,Regular Season,1,09/08,Away,MIN,L,...,85.8,2,24,12.0,0,--,--,atl,20.56,8000
7,mattryan/310,"Ryan, Matt",QB,2019,Regular Season,3,09/22,Away,IND,L,...,121.1,2,3,1.5,0,--,--,atl,23.46,7800
8,mattryan/310,"Ryan, Matt",QB,2019,Regular Season,5,10/06,Away,HOU,L,...,102.6,3,7,2.3,1,--,--,atl,32.90,7700
9,mattryan/310,"Ryan, Matt",QB,2019,Regular Season,6,10/13,Away,ARI,L,...,144.9,2,7,3.5,0,1,0,atl,30.94,8100


In [4]:
qbX_Y = pd.merge(right=  basicQBStats, left=gameY,how='inner')

In [5]:
sum =0
for col in qbX_Y.columns:
    sum += qbX_Y[col].isna().sum()

In [6]:
sum

114

In [10]:
qbX_Y['Years Played'].isna().sum() # drop years played.
qbX_Y = qbX_Y.drop(['Years Played'],axis=1)



KeyError: 'Years Played'

In [16]:
sum =0
for col in qbX_Y.columns:
    sum += qbX_Y[col].isna().sum()
sum
qbX_Y['points']

0      33.56
1      14.44
2      23.32
3      25.84
4      17.52
5       0.00
6      20.56
7      23.46
8      32.90
9      30.94
10      0.80
11      0.00
12     -0.20
13     -0.20
14      5.36
15     16.12
16     16.72
17     21.16
18     21.00
19     27.60
20     12.14
21      9.08
22     20.72
23     18.06
24      4.64
25     14.60
26     19.54
27     21.54
28      7.84
29     14.38
       ...  
73     28.90
74     15.68
75      8.58
76      9.30
77      0.96
78      0.00
79     16.50
80     17.34
81      1.16
82     21.58
83     13.30
84     10.44
85     16.16
86     18.90
87     25.02
88     24.34
89      3.28
90     17.96
91     22.22
92     18.46
93     12.92
94      9.42
95     13.22
96     21.34
97     16.00
98     16.78
99     24.66
100     4.70
101    24.92
102    12.96
Name: points, Length: 103, dtype: float64

In [110]:
from scipy.stats import  tmean
tmean(qbX_Y['points'])

portfolio = pd.DataFrame()
portfolio['mean']=qbX_Y['points'].rolling(5,win_type='boxcar', min_periods=1).mean()
portfolio['std'] = qbX_Y['points'].rolling(5, min_periods=1).std()#.dropna()
portfolio['variance']=qbX_Y['points'].rolling(5, min_periods=1).var()#.dropna()

#we maximize the maximum cost
portfolio['cost'] = qbX_Y['salary'].rolling(5, min_periods=1).max()

portfolio['player']=qbX_Y['Player Id']
p=portfolio.groupby('player')
p.median()
#p.mean()

def predict(DF,position, windowsize=5 ):
	portfolio = pd.DataFrame()
	#expected value	
	portfolio['mean'] = DF['points'].rolling(windowsize,win_type='boxcar', min_periods=1).mean()
	#standard deviation
	portfolio['std'] = DF['points'].rolling(windowsize, min_periods=1).std()
	#variance
	portfolio['variance'] = DF['points'].rolling(windowsize, min_periods=1).var()
	#X identifier
	portfolio['player'] = DF['Player Id']
	portfolio['position'] = pd.Series([position]*len(portfolio['player']))
	portfolio['cost'] = DF['salary'].rolling(5, min_periods=1).max()

	return portfolio

predict(qbX_Y,1).dropna()

,mean,std,variance,player,position,cost
1,24.000000,13.519882,182.787200,lamarjackson/2560757,1,8300.0
2,23.773333,9.568058,91.547733,lamarjackson/2560757,1,8400.0
3,24.290000,7.880330,62.099600,deshaunwatson/2558063,1,8400.0
4,22.936000,7.466008,55.741280,deshaunwatson/2558063,1,8400.0
5,16.224000,10.135348,102.725280,blakebortles/2543477,1,8400.0
6,17.448000,10.235093,104.757120,mattryan/310,1,8400.0
7,17.476000,10.255344,105.172080,mattryan/310,1,8300.0
8,18.888000,12.027706,144.665720,mattryan/310,1,8300.0
9,21.572000,13.096019,171.505720,mattryan/310,1,8100.0
10,21.732000,12.767338,163.004920,lukefalk/2560720,1,8100.0


In [123]:
Ystats = pd.read_csv('PK.csv', index_col=0)

Ystats['salary'].max()

0